In [5]:
pip install -U langchain-community faiss-cpu langchain-openai tiktoken langchain-text-splitters

3905.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
  Using cached langchain_community-0.2.4-py3-none-any.whl.metadata (2.4 kB)
Using cached langchain_community-0.2.4-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.4
    Uninstalling langchain-0.1.4:
      Successfully uninstalled langchain-0.1.4
  At

In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
### LLM

local_llm = "llama3"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

doc = PyPDFLoader().load_document("")

text_splitter = RecursiveCharacterTextSplitter().from_tiktoken_encoder(chunk_size=4096,chunk_overlap=0)
pdf_loader = PyPDFLoader(text_splitter=text_splitter)

